In [1]:
import pandas as pd

In [2]:
import torch
import numpy as np
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer
from transformers import AutoModelForMultipleChoice, TrainingArguments, Trainer
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from torch.utils.data import DataLoader

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [3]:
# exp004
import dataclasses
import torch
import pandas as pd
from sklearn.model_selection import KFold
from transformers import AutoModel, AutoTokenizer, AutoModelForMultipleChoice
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from transformers import AutoModelForMultipleChoice, TrainingArguments, Trainer
from datasets import Dataset
from typing import Optional, Union

import transformers
import wandb
from datetime import datetime as dt
import os
import numpy as np
import tqdm

import logging 
from logging import Logger



@dataclasses.dataclass
class BertConfig:
    
    experiment_name: str
    dataset_dir: str
    
    debug: bool = False

    lr: float = 1e-5
    model_name: str = "OpenAssistant/reward-model-deberta-v3-large-v2"
    num_context: int = 3
    max_length: int = 512
    batch_size: int = 2
    epochs: int = 10
    iters_to_accumlate: int = 8
    weight_decay: float = 0.01
    warmup_ratio: float = 0.1
    
    freeze_embeddings: bool = True
    freeze_layers: int = 18
    reinitialize_layers: int = 0
    
    assume_completely_retrieved: bool = False
    n_samples: int = None
    steps: int = 100
    
    lora_r: float = 2
    lora_alpha: float = 4
    lora_dropout: float = 0.1
    use_peft: bool = False

def get_logger(
    output_dir: str,
):
    """
    logger を作成する. formatter は "%Y-%m-%d %H:%M:%S" で作成する.
    """
    logger = logging.getLogger(__name__)
    logger.setLevel(logging.DEBUG)
    
    # formatter
    formatter = logging.Formatter("%(asctime)s %(levelname)s %(message)s")
    
    # handler
    handler = logging.StreamHandler()
    handler.setLevel(logging.DEBUG)
    handler.setFormatter(formatter)
    logger.addHandler(handler)
    
    handler = logging.FileHandler(f"{output_dir}/log.txt", "w")
    handler.setLevel(logging.DEBUG)
    handler.setFormatter(formatter)
    logger.addHandler(handler)
    
    return logger


@dataclasses.dataclass
class DataCollatorForMultipleChoice:
    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    
    def __call__(self, features):
        label_name = 'label' if 'label' in features[0].keys() else 'labels'
        labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]['input_ids'])
        flattened_features = [
            [{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features
        ]
        flattened_features = sum(flattened_features, [])
        
        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors='pt',
        )
        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        batch['labels'] = torch.tensor(labels, dtype=torch.int64)
        return batch


        
def preprocess_df(df, config):
    df["context"] = ""
    for i in range(config.num_context):
        df["context"] += df[f"searched_wiki_id_{i}"].astype(str) + "\n\n"
    
    for col in ["A", "B", "C", "D", "E"]:
        df[col] = df[col].fillna("")
    return df[["prompt", "context", "A", "B", "C", "D", "E", "answer"]]


def map_at_3(predictions, labels):
    map_sum = 0
    pred = np.argsort(-1*np.array(predictions),axis=1)[:,:3]
    for x,y in zip(pred,labels):
        z = [1/i if y==j else 0 for i,j in zip([1,2,3],x)]
        map_sum += np.sum(z)
    return map_sum / len(predictions)

def compute_metrics(p):
    predictions = p.predictions.tolist()
    labels = p.label_ids.tolist()
    return {"map@3": map_at_3(predictions, labels)}


In [4]:
df_test = pd.read_parquet("../output/context_pipeline/stage1/exp009.py/20230922162941_gte-base_wikiall_without_sep_targetprompt_and_choice_without_sep_token_length120_stride_sentence4_drop_categoryTrue_all/valid.parquet")

In [5]:
option_to_index = {option: idx for idx, option in enumerate('ABCDE')}
label = df_test["answer"].map(option_to_index).values

In [6]:
df_test = df_test.iloc[:, 1:]

In [7]:
logger = get_logger(output_dir="")
logger.info("load data")

2023-09-25 08:09:54,164 INFO load data


In [8]:
def calc_map3(
    model_name,
    max_length,
    num_content,
):
    df_test = pd.read_parquet("../output/context_pipeline/stage1/exp009.py/20230922162941_gte-base_wikiall_without_sep_targetprompt_and_choice_without_sep_token_length120_stride_sentence4_drop_categoryTrue_all/valid.parquet")
    option_to_index = {option: idx for idx, option in enumerate('ABCDE')}
    label = df_test["answer"].map(option_to_index).values
    logger = get_logger(output_dir="")
    logger.info("load data")
    
    config = BertConfig(
        debug=False,
        batch_size=1,
        experiment_name=f"",
        dataset_dir="",
        model_name=model_name,
        max_length=max_length,
        num_context=num_content,
    )
    df_test = preprocess_df(df_test, config)
    df_test = df_test[df_test["answer"].isin(["A", "B", "C", "D", "E"])]
    
    tokenizer = AutoTokenizer.from_pretrained(config.model_name)
    option_to_index = {option: idx for idx, option in enumerate('ABCDE')}
    index_to_option = {v: k for k,v in option_to_index.items()}
    def preprocess(example):
        first_sentence = [ "[CLS] " + example['context'] ] * 5
        second_sentences = [" #### " + example['prompt'] + " [SEP] " + example[option] + " [SEP]" for option in 'ABCDE']
        tokenized_example = tokenizer(first_sentence, second_sentences, truncation="only_first", 
                                      max_length=config.max_length, add_special_tokens=False)
        tokenized_example['label'] = option_to_index[example['answer']]
        return tokenized_example

    test_dataset = Dataset.from_pandas(df_test)
    tokenized_test_dataset = test_dataset.map(preprocess, remove_columns=["prompt", "context", "A", "B", "C", "D", "E", "answer", "__index_level_0__"])

    data_collator = DataCollatorForMultipleChoice(tokenizer=tokenizer)
    test_dataloader = DataLoader(tokenized_test_dataset, batch_size=1, shuffle=False, collate_fn=data_collator)
    
    model = AutoModelForMultipleChoice.from_pretrained(config.model_name).cuda()
    
    test_predictions = []
    for batch in tqdm.tqdm(test_dataloader):
        with torch.autocast("cuda"):
            for k in batch.keys():
                batch[k] = batch[k].cuda()
            with torch.no_grad():
                outputs = model(**batch)
            test_predictions.append(outputs.logits.cpu().detach())

    test_predictions = torch.cat(test_predictions)
    test_predictions = test_predictions.numpy()
    
    print(map_at_3(test_predictions, label))
    return {
        "model_name": model_name,
        "max_length": max_length,
        "num_content": num_content,
        "map3": map_at_3(test_predictions, label),
        "predictions": test_predictions
    }

In [9]:
ret = []

In [11]:
ret.append(calc_map3(
    model_name="../output/stage2/exp005.py/20230923195407_new_data_all300val_maxlen256/fold0/",
    max_length=256,
    num_content=2,
))

2023-09-25 08:11:09,200 INFO load data
2023-09-25 08:11:09,200 INFO load data
2023-09-25 08:11:09,200 INFO load data


  0%|          | 0/3900 [00:00<?, ?ex/s]

100%|█████████████████████████████████████| 3900/3900 [03:25<00:00, 18.96it/s]


0.867008547008549


In [12]:
ret.append(calc_map3(
    model_name="../output/stage2/exp005.py/20230923195407_new_data_all300val_maxlen256/fold0/",
    max_length=384,
    num_content=3,
))

2023-09-25 08:14:56,888 INFO load data
2023-09-25 08:14:56,888 INFO load data
2023-09-25 08:14:56,888 INFO load data
2023-09-25 08:14:56,888 INFO load data


  0%|          | 0/3900 [00:00<?, ?ex/s]

100%|█████████████████████████████████████| 3900/3900 [03:42<00:00, 17.51it/s]


0.8694444444444464


In [13]:
ret.append(calc_map3(
    model_name="../output/stage2/exp005.py/20230923195407_new_data_all300val_maxlen256/fold0/",
    max_length=512,
    num_content=4,
))

2023-09-25 08:19:06,054 INFO load data
2023-09-25 08:19:06,054 INFO load data
2023-09-25 08:19:06,054 INFO load data
2023-09-25 08:19:06,054 INFO load data
2023-09-25 08:19:06,054 INFO load data


  0%|          | 0/3900 [00:00<?, ?ex/s]

100%|█████████████████████████████████████| 3900/3900 [04:31<00:00, 14.36it/s]


0.8678632478632493


In [14]:
ret.append(calc_map3(
    model_name="../output/stage2/exp005.py/20230923195407_new_data_all300val_maxlen256/fold0/",
    max_length=768,
    num_content=7,
))

2023-09-25 08:24:09,735 INFO load data
2023-09-25 08:24:09,735 INFO load data
2023-09-25 08:24:09,735 INFO load data
2023-09-25 08:24:09,735 INFO load data
2023-09-25 08:24:09,735 INFO load data
2023-09-25 08:24:09,735 INFO load data


  0%|          | 0/3900 [00:00<?, ?ex/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3900/3900 [09:42<00:00,  6.70it/s]


0.869102564102566


In [15]:
ret.append(calc_map3(
    model_name="../output/stage2/exp005.py/20230924021914_new_data_all300val_maxlen384/fold0/",
    max_length=384,
    num_content=3,
))

2023-09-25 08:34:36,183 INFO load data
2023-09-25 08:34:36,183 INFO load data
2023-09-25 08:34:36,183 INFO load data
2023-09-25 08:34:36,183 INFO load data
2023-09-25 08:34:36,183 INFO load data
2023-09-25 08:34:36,183 INFO load data
2023-09-25 08:34:36,183 INFO load data


  0%|          | 0/3900 [00:00<?, ?ex/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3900/3900 [03:51<00:00, 16.81it/s]


0.8711111111111133


In [16]:
ret.append(calc_map3(
    model_name="../output/stage2/exp005.py/20230924021914_new_data_all300val_maxlen384/fold0/",
    max_length=512,
    num_content=4,
))

2023-09-25 08:38:56,083 INFO load data
2023-09-25 08:38:56,083 INFO load data
2023-09-25 08:38:56,083 INFO load data
2023-09-25 08:38:56,083 INFO load data
2023-09-25 08:38:56,083 INFO load data
2023-09-25 08:38:56,083 INFO load data
2023-09-25 08:38:56,083 INFO load data
2023-09-25 08:38:56,083 INFO load data


  0%|          | 0/3900 [00:00<?, ?ex/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3900/3900 [04:35<00:00, 14.16it/s]


0.8695726495726515


In [17]:
ret.append(calc_map3(
    model_name="../output/stage2/exp005.py/20230924021914_new_data_all300val_maxlen384/fold0/",
    max_length=768,
    num_content=7,
))

2023-09-25 08:44:05,819 INFO load data
2023-09-25 08:44:05,819 INFO load data
2023-09-25 08:44:05,819 INFO load data
2023-09-25 08:44:05,819 INFO load data
2023-09-25 08:44:05,819 INFO load data
2023-09-25 08:44:05,819 INFO load data
2023-09-25 08:44:05,819 INFO load data
2023-09-25 08:44:05,819 INFO load data
2023-09-25 08:44:05,819 INFO load data


  0%|          | 0/3900 [00:00<?, ?ex/s]

100%|█████████████████████████████████████| 3900/3900 [09:18<00:00,  6.98it/s]


0.8690598290598311


In [18]:
ret.append(calc_map3(
    model_name="../output/stage2/exp005.py/20230924084819_new_data_all300val_maxlen512/fold0/",
    max_length=512,
    num_content=4,
))

2023-09-25 08:54:12,453 INFO load data
2023-09-25 08:54:12,453 INFO load data
2023-09-25 08:54:12,453 INFO load data
2023-09-25 08:54:12,453 INFO load data
2023-09-25 08:54:12,453 INFO load data
2023-09-25 08:54:12,453 INFO load data
2023-09-25 08:54:12,453 INFO load data
2023-09-25 08:54:12,453 INFO load data
2023-09-25 08:54:12,453 INFO load data
2023-09-25 08:54:12,453 INFO load data


  0%|          | 0/3900 [00:00<?, ?ex/s]

100%|█████████████████████████████████████| 3900/3900 [04:26<00:00, 14.62it/s]


0.8726923076923098


In [19]:
ret.append(calc_map3(
    model_name="../output/stage2/exp005.py/20230924084819_new_data_all300val_maxlen512/fold0/",
    max_length=768,
    num_content=7,
))

2023-09-25 08:59:20,196 INFO load data
2023-09-25 08:59:20,196 INFO load data
2023-09-25 08:59:20,196 INFO load data
2023-09-25 08:59:20,196 INFO load data
2023-09-25 08:59:20,196 INFO load data
2023-09-25 08:59:20,196 INFO load data
2023-09-25 08:59:20,196 INFO load data
2023-09-25 08:59:20,196 INFO load data
2023-09-25 08:59:20,196 INFO load data
2023-09-25 08:59:20,196 INFO load data
2023-09-25 08:59:20,196 INFO load data


  0%|          | 0/3900 [00:00<?, ?ex/s]

100%|█████████████████████████████████████| 3900/3900 [09:03<00:00,  7.17it/s]


0.8721794871794889


In [21]:
df = pd.DataFrame(ret)

In [22]:
df

,model_name,max_length,num_content,map3,predictions
0,../output/stage2/exp005.py/20230923195407_new_...,256,2,0.867009,"[[-1.337, 1.047, 8.664, -2.17, 1.1875], [-6.95..."
1,../output/stage2/exp005.py/20230923195407_new_...,384,3,0.869444,"[[-2.62, -1.216, 8.02, -3.955, -0.3167], [-6.5..."
2,../output/stage2/exp005.py/20230923195407_new_...,512,4,0.867863,"[[-1.972, 0.662, 8.08, -3.24, 0.0765], [-6.13,..."
3,../output/stage2/exp005.py/20230923195407_new_...,768,7,0.869103,"[[-0.787, 2.795, 5.69, -1.655, 1.073], [-7.145..."
4,../output/stage2/exp005.py/20230924021914_new_...,384,3,0.871111,"[[-1.577, -1.061, 8.01, -3.428, -0.8506], [-6...."
5,../output/stage2/exp005.py/20230924021914_new_...,512,4,0.869573,"[[-1.027, 0.09576, 8.16, -3.041, -0.313], [-6...."
6,../output/stage2/exp005.py/20230924021914_new_...,768,7,0.869060,"[[-0.3408, 1.863, 5.83, -2.057, 0.4429], [-7.5..."
7,../output/stage2/exp005.py/20230924084819_new_...,512,4,0.872692,"[[0.7046, 2.053, 9.43, -0.767, 1.345], [-6.453..."
8,../output/stage2/exp005.py/20230924084819_new_...,768,7,0.872179,"[[0.7993, 3.625, 7.414, -0.2031, 2.418], [-8.2..."


In [27]:
test_predictions

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 test_predictions                                                                             │
│   2                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'test_predictions' is not defined

In [36]:
np.stack(df_test_["pred"].values)

array([[-1.337 ,  1.047 ,  8.664 , -2.17  ,  1.1875],
       [-6.957 , -7.453 ,  4.77  , -7.746 , -7.363 ],
       [-2.693 , -7.727 , -8.05  , -1.409 , -6.516 ],
       ...,
       [ 1.574 , -1.571 , -1.151 , -2.879 , -7.63  ],
       [-2.307 , -0.715 , -2.492 , -1.547 , -5.465 ],
       [-6.316 ,  1.197 , -3.371 ,  4.277 , -9.086 ]], dtype=float16)

In [37]:
ret2 = []
for i in range(len(df)):
    series = df.iloc[i]
    df_test_ = df_test.copy()
    df_test_["pred"] = [x for x in series["predictions"]]
    
    for source, w_df in df_test_.groupby("source"):
        print(source)
        option_to_index = {option: idx for idx, option in enumerate('ABCDE')}
        label = w_df["answer"].map(option_to_index).values
        test_predictions = np.stack(w_df["pred"].values)
        ret2.append({
            "model_name": series["model_name"],
            "max_length": series["max_length"],
            "num_content": series["num_content"],
            "source": source,
            "map3": map_at_3(test_predictions, label)
        })

1
11
2
3
4
5
6
7
8
9
additional_data/MMLU/test.csv
additional_data/OpenBookQA-V1-Sep2018/OpenBookQA-V1-Sep2018/Data/Main/train.tsv
additional_data/ScienceQA/test.parquet
1
11
2
3
4
5
6
7
8
9
additional_data/MMLU/test.csv
additional_data/OpenBookQA-V1-Sep2018/OpenBookQA-V1-Sep2018/Data/Main/train.tsv
additional_data/ScienceQA/test.parquet
1
11
2
3
4
5
6
7
8
9
additional_data/MMLU/test.csv
additional_data/OpenBookQA-V1-Sep2018/OpenBookQA-V1-Sep2018/Data/Main/train.tsv
additional_data/ScienceQA/test.parquet
1
11
2
3
4
5
6
7
8
9
additional_data/MMLU/test.csv
additional_data/OpenBookQA-V1-Sep2018/OpenBookQA-V1-Sep2018/Data/Main/train.tsv
additional_data/ScienceQA/test.parquet
1
11
2
3
4
5
6
7
8
9
additional_data/MMLU/test.csv
additional_data/OpenBookQA-V1-Sep2018/OpenBookQA-V1-Sep2018/Data/Main/train.tsv
additional_data/ScienceQA/test.parquet
1
11
2
3
4
5
6
7
8
9
additional_data/MMLU/test.csv
additional_data/OpenBookQA-V1-Sep2018/OpenBookQA-V1-Sep2018/Data/Main/train.tsv
additional_data/Sci

In [38]:
df_ret2 = pd.DataFrame(ret2)

In [42]:
df_ret2.sort_values(["model_name", "source", "max_length"])

,model_name,max_length,num_content,source,map3
0,../output/stage2/exp005.py/20230923195407_new_...,256,2,1,0.857222
13,../output/stage2/exp005.py/20230923195407_new_...,384,3,1,0.873889
26,../output/stage2/exp005.py/20230923195407_new_...,512,4,1,0.863889
39,../output/stage2/exp005.py/20230923195407_new_...,768,7,1,0.863889
1,../output/stage2/exp005.py/20230923195407_new_...,256,2,11,0.965000
...,...,...,...,...,...
114,../output/stage2/exp005.py/20230924084819_new_...,768,7,additional_data/MMLU/test.csv,0.743333
102,../output/stage2/exp005.py/20230924084819_new_...,512,4,additional_data/OpenBookQA-V1-Sep2018/OpenBook...,0.937222
115,../output/stage2/exp005.py/20230924084819_new_...,768,7,additional_data/OpenBookQA-V1-Sep2018/OpenBook...,0.937222
103,../output/stage2/exp005.py/20230924084819_new_...,512,4,additional_data/ScienceQA/test.parquet,0.932778


In [49]:
df.to_pickle("20230925_ret.pickle")
df_ret2.to_pickle("20230925_ret2.pickle")

In [50]:
df_ret2.pivot_table(columns=["model_name", "source"], index="max_length", values="map3")

model_name ../output/stage2/exp005.py/20230923195407_new_data_all300val_maxlen256/fold0/  \
source                                                                                 1   
max_length                                                                                 
256                                                  0.857222                              
384                                                  0.873889                              
512                                                  0.863889                              
768                                                  0.863889                              

model_name                                                              \
source            11         2         3         4         5         6   
max_length                                                               
256         0.965000  0.875556  0.830556  0.863889  0.881111  0.875000   
384         0.960556  0.883333  0.822778  0.880000  0.871111  0.884444   
512         0.961111  0.879444  0.824444  0.881667  0.853333  0.877778   
768         0.968333  0.876111  0.826111  0.887222  0.852778  0.875000   

model_name                                ...  \
source             7         8         9  ...   
max_length                                ...   
256         0.844444  0.850000  0.842222  ...   
384         0.856667  0.847222  0.838333  ...   
512         0.866111  0.850556  0.843889  ...   
768         0.866111  0.862778  0.840000  ...   

model_name ../output/stage2/exp005.py/20230924084819_new_data_all300val_maxlen512/fold0/  \
source                                                                                 3   
max_length                                                                                 
256                                                       NaN                              
384                                                       NaN                              
512                                                  0.823889                              
768                                                  0.817778                              

model_name                                                              \
source             4         5         6         7         8         9   
max_length                                                               
256              NaN       NaN       NaN       NaN       NaN       NaN   
384              NaN       NaN       NaN       NaN       NaN       NaN   
512         0.877222  0.877778  0.881667  0.862778  0.858889  0.831111   
768         0.883333  0.867778  0.882222  0.851111  0.866111  0.833889   

model_name                                \
source     additional_data/MMLU/test.csv   
max_length                                 
256                                  NaN   
384                                  NaN   
512                             0.738333   
768                             0.743333   

model_name                                                                                  \
source     additional_data/OpenBookQA-V1-Sep2018/OpenBookQA-V1-Sep2018/Data/Main/train.tsv   
max_length                                                                                   
256                                                       NaN                                
384                                                       NaN                                
512                                                  0.937222                                
768                                                  0.937222                                

model_name                                         
source     additional_data/ScienceQA/test.parquet  
max_length                                         
256                                           NaN  
384                                           NaN  
512                                      0.932778  
768                                      0.93666

In [51]:
df_ret2.pivot_table(index=["model_name", "max_length"], columns="source", values="map3")

source                                                                1  \
model_name                                         max_length             
../output/stage2/exp005.py/20230923195407_new_d... 256         0.857222   
                                                   384         0.873889   
                                                   512         0.863889   
                                                   768         0.863889   
../output/stage2/exp005.py/20230924021914_new_d... 384         0.860556   
                                                   512         0.862778   
                                                   768         0.866667   
../output/stage2/exp005.py/20230924084819_new_d... 512         0.866111   
                                                   768         0.871667   

source                                                               11  \
model_name                                         max_length             
../output/stage2/exp005.py/20230923195407_new_d... 256         0.965000   
                                                   384         0.960556   
                                                   512         0.961111   
                                                   768         0.968333   
../output/stage2/exp005.py/20230924021914_new_d... 384         0.958333   
                                                   512         0.957778   
                                                   768         0.968333   
../output/stage2/exp005.py/20230924084819_new_d... 512         0.967778   
                                                   768         0.970000   

source                                                                2  \
model_name                                         max_length             
../output/stage2/exp005.py/20230923195407_new_d... 256         0.875556   
                                                   384         0.883333   
                                                   512         0.879444   
                                                   768         0.876111   
../output/stage2/exp005.py/20230924021914_new_d... 384         0.885000   
                                                   512         0.886667   
                                                   768         0.875556   
../output/stage2/exp005.py/20230924084819_new_d... 512         0.889444   
                                                   768         0.877222   

source                                                                3  \
model_name                                         max_length             
../output/stage2/exp005.py/20230923195407_new_d... 256         0.830556   
                                                   384         0.822778   
                                                   512         0.824444   
                                                   768         0.826111   
../output/stage2/exp005.py/20230924021914_new_d... 384         0.836667   
                                                   512         0.833889   
                                                   768         0.811111   
../output/stage2/exp005.py/20230924084819_new_d... 512         0.823889   
                                                   768         0.817778   

source                                                                4  \
model_name                                         max_length             
../output/stage2/exp005.py/20230923195407_new_d... 256         0.863889   
                                                   384         0.880000   
                                                   512         0.881667   
                                                   768         0.887222   
../output/stage2/exp005.py/20230924021914_new_d... 384         0.880556   
                                                   512         0.882222   
                                                   768         0.890556   
../output/stage2/exp0

In [55]:
df_ret2[df_ret2["source"].isin(["1", "2", "3", "4", "5", "6", "7", "8"])].groupby(["model_name", "max_length"])["map3"].mean()

model_name                                                                     max_length
../output/stage2/exp005.py/20230923195407_new_data_all300val_maxlen256/fold0/  256           0.859722
                                                                               384           0.864931
                                                                               512           0.862153
                                                                               768           0.863750
../output/stage2/exp005.py/20230924021914_new_data_all300val_maxlen384/fold0/  384           0.867014
                                                                               512           0.865417
                                                                               768           0.863264
../output/stage2/exp005.py/20230924084819_new_data_all300val_maxlen512/fold0/  512           0.867222
                                                                               768           0

In [56]:
df_ret2.groupby(["model_name", "max_length"])["map3"].mean()

model_name                                                                     max_length
../output/stage2/exp005.py/20230923195407_new_data_all300val_maxlen256/fold0/  256           0.867009
                                                                               384           0.869444
                                                                               512           0.867863
                                                                               768           0.869103
../output/stage2/exp005.py/20230924021914_new_data_all300val_maxlen384/fold0/  384           0.871111
                                                                               512           0.869573
                                                                               768           0.869060
../output/stage2/exp005.py/20230924084819_new_data_all300val_maxlen512/fold0/  512           0.872692
                                                                               768           0

In [72]:
df_test[df_test["A"].apply(len) < 30]["source"].value_counts()

11                                                                                 273
additional_data/OpenBookQA-V1-Sep2018/OpenBookQA-V1-Sep2018/Data/Main/train.tsv    261
9                                                                                  215
additional_data/ScienceQA/test.parquet                                             184
additional_data/MMLU/test.csv                                                      180
8                                                                                  143
7                                                                                  132
4                                                                                  100
3                                                                                   91
2                                                                                   69
5                                                                                   67
1                                          

In [69]:
ret3 = []
for i in range(len(df)):
    series = df.iloc[i]
    df_test_ = df_test.copy()
    df_test_["pred"] = [x for x in series["predictions"]]
    df_test_ = df_test_[df_test_["A"].apply(len) < 20]
    
    for source, w_df in df_test_.groupby("source"):
        print(source)
        option_to_index = {option: idx for idx, option in enumerate('ABCDE')}
        label = w_df["answer"].map(option_to_index).values
        test_predictions = np.stack(w_df["pred"].values)
        ret3.append({
            "model_name": series["model_name"],
            "max_length": series["max_length"],
            "num_content": series["num_content"],
            "source": source,
            "map3": map_at_3(test_predictions, label)
        })

1
11
2
3
4
5
6
7
8
9
additional_data/MMLU/test.csv
additional_data/OpenBookQA-V1-Sep2018/OpenBookQA-V1-Sep2018/Data/Main/train.tsv
additional_data/ScienceQA/test.parquet
1
11
2
3
4
5
6
7
8
9
additional_data/MMLU/test.csv
additional_data/OpenBookQA-V1-Sep2018/OpenBookQA-V1-Sep2018/Data/Main/train.tsv
additional_data/ScienceQA/test.parquet
1
11
2
3
4
5
6
7
8
9
additional_data/MMLU/test.csv
additional_data/OpenBookQA-V1-Sep2018/OpenBookQA-V1-Sep2018/Data/Main/train.tsv
additional_data/ScienceQA/test.parquet
1
11
2
3
4
5
6
7
8
9
additional_data/MMLU/test.csv
additional_data/OpenBookQA-V1-Sep2018/OpenBookQA-V1-Sep2018/Data/Main/train.tsv
additional_data/ScienceQA/test.parquet
1
11
2
3
4
5
6
7
8
9
additional_data/MMLU/test.csv
additional_data/OpenBookQA-V1-Sep2018/OpenBookQA-V1-Sep2018/Data/Main/train.tsv
additional_data/ScienceQA/test.parquet
1
11
2
3
4
5
6
7
8
9
additional_data/MMLU/test.csv
additional_data/OpenBookQA-V1-Sep2018/OpenBookQA-V1-Sep2018/Data/Main/train.tsv
additional_data/Sci

In [70]:
df_ret3 = pd.DataFrame(ret3)

In [71]:
df_ret3.pivot_table(index=["model_name", "max_length"], columns="source", values="map3")

source                                                                1  \
model_name                                         max_length             
../output/stage2/exp005.py/20230923195407_new_d... 256         0.913194   
                                                   384         0.916667   
                                                   512         0.906250   
                                                   768         0.909722   
../output/stage2/exp005.py/20230924021914_new_d... 384         0.909722   
                                                   512         0.899306   
                                                   768         0.920139   
../output/stage2/exp005.py/20230924084819_new_d... 512         0.913194   
                                                   768         0.909722   

source                                                               11  \
model_name                                         max_length             
../output/stage2/exp005.py/20230923195407_new_d... 256         0.959302   
                                                   384         0.958010   
                                                   512         0.958656   
                                                   768         0.965116   
../output/stage2/exp005.py/20230924021914_new_d... 384         0.955426   
                                                   512         0.955426   
                                                   768         0.965116   
../output/stage2/exp005.py/20230924084819_new_d... 512         0.966408   
                                                   768         0.967054   

source                                                                2  \
model_name                                         max_length             
../output/stage2/exp005.py/20230923195407_new_d... 256         0.833333   
                                                   384         0.884615   
                                                   512         0.852564   
                                                   768         0.868590   
../output/stage2/exp005.py/20230924021914_new_d... 384         0.846154   
                                                   512         0.865385   
                                                   768         0.875000   
../output/stage2/exp005.py/20230924084819_new_d... 512         0.858974   
                                                   768         0.865385   

source                                                                3  \
model_name                                         max_length             
../output/stage2/exp005.py/20230923195407_new_d... 256         0.813853   
                                                   384         0.831169   
                                                   512         0.826840   
                                                   768         0.846320   
../output/stage2/exp005.py/20230924021914_new_d... 384         0.813853   
                                                   512         0.820346   
                                                   768         0.820346   
../output/stage2/exp005.py/20230924084819_new_d... 512         0.826840   
                                                   768         0.865801   

source                                                                4  \
model_name                                         max_length             
../output/stage2/exp005.py/20230923195407_new_d... 256         0.897287   
                                                   384         0.918605   
                                                   512         0.916667   
                                                   768         0.922481   
../output/stage2/exp005.py/20230924021914_new_d... 384         0.918605   
                                                   512         0.926357   
                                                   768         0.926357   
../output/stage2/exp0